In [1]:
import xgboost as xgb
print(xgb.__version__)

3.0.4


In [2]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

/Users/venkatj/Projects/Regression_ML_EndtoEnd/.venv/bin/python
3.0.4
/Users/venkatj/Projects/Regression_ML_EndtoEnd/.venv/lib/python3.11/site-packages/xgboost/__init__.py


In [3]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

/Users/venkatj/Projects/Regression_ML_EndtoEnd/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv("/Users/venkatj/Projects/Regression_ML_EndtoEnd/data/processed/feature_engineered_train.csv")
eval_df  = pd.read_csv("/Users/venkatj/Projects/Regression_ML_EndtoEnd/data/processed/feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (547808, 39)
Eval shape: (140986, 39)


In [6]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [8]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("/Users/venkatj/Projects/Regression_ML_EndtoEnd/mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

2025/11/26 10:12:07 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2025-11-26 10:12:07,502] A new study created in memory with name: no-name-56c70bc2-dcba-4ce4-b391-a31022c2b40b
[I 2025-11-26 10:12:12,333] Trial 0 finished with value: 73108.5049279825 and parameters: {'n_estimators': 807, 'max_depth': 3, 'learning_rate': 0.15370596230074926, 'subsample': 0.8091191460758946, 'colsample_bytree': 0.6115366983799251, 'min_child_weight': 6, 'gamma': 2.204702641448533, 'reg_alpha': 0.016656590050111716, 'reg_lambda': 7.598225207097269e-08}. Best is trial 0 with value: 73108.5049279825.
[I 2025-11-26 10:12:17,435] Trial 1 finished with value: 71570.9424638174 and parameters: {'n_estimators': 222, 'max_depth': 10, 'learning_rate': 0.02766953213103971, 'subsample': 0.6767736143103711, 'colsample_bytree': 0.7551213103471712, 'min_child_weight': 4, 'gamma': 0.3347855223948726, 'reg_alpha': 0.0024359147950194855, 'reg_lambda

Best params: {'n_estimators': 475, 'max_depth': 7, 'learning_rate': 0.07381555706418874, 'subsample': 0.998133843510743, 'colsample_bytree': 0.6298501615049712, 'min_child_weight': 1, 'gamma': 4.177058273397941, 'reg_alpha': 7.767030395698796e-06, 'reg_lambda': 9.494920735500862}


In [9]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 31366.986767611008
RMSE: 68709.19242465877
R²: 0.9647540339175688


2025/11/26 10:13:57 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/11/26 10:13:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
